In [1]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [2]:
dossier="/Github/eGloss"
enonce="Simple"
theme="SeparationMorphes"
adaptatifPenalite="0.1000000"
dernierIndiceNote=75.0
msgPlusDeMots=u"Il y a plus de mots"
msgMoinsDeMots=u"Il y a moins de mots"
def nbIndiceNotes(nb):
    if nb==1: 
        return ["%50%"]
    elif nb==2:
        return ["","%50%"]
    else:
        return [""]+["%"+str(int((i+2)*dernierIndiceNote/nb))+"%" for i in range(nb-1)]   

In [3]:
for i in range(20):
    print i,nbIndiceNotes(i) 

0 ['']
1 ['%50%']
2 ['', '%50%']
3 ['', '%50%', '%75%']
4 ['', '%37%', '%56%', '%75%']
5 ['', '%30%', '%45%', '%60%', '%75%']
6 ['', '%25%', '%37%', '%50%', '%62%', '%75%']
7 ['', '%21%', '%32%', '%42%', '%53%', '%64%', '%75%']
8 ['', '%18%', '%28%', '%37%', '%46%', '%56%', '%65%', '%75%']
9 ['', '%16%', '%25%', '%33%', '%41%', '%50%', '%58%', '%66%', '%75%']
10 ['', '%15%', '%22%', '%30%', '%37%', '%45%', '%52%', '%60%', '%67%', '%75%']
11 ['', '%13%', '%20%', '%27%', '%34%', '%40%', '%47%', '%54%', '%61%', '%68%', '%75%']
12 ['', '%12%', '%18%', '%25%', '%31%', '%37%', '%43%', '%50%', '%56%', '%62%', '%68%', '%75%']
13 ['', '%11%', '%17%', '%23%', '%28%', '%34%', '%40%', '%46%', '%51%', '%57%', '%63%', '%69%', '%75%']
14 ['', '%10%', '%16%', '%21%', '%26%', '%32%', '%37%', '%42%', '%48%', '%53%', '%58%', '%64%', '%69%', '%75%']
15 ['', '%10%', '%15%', '%20%', '%25%', '%30%', '%35%', '%40%', '%45%', '%50%', '%55%', '%60%', '%65%', '%70%', '%75%']
16 ['', '%9%', '%14%', '%18%', '%23%',

nbIndiceNotes={
    1:["%50%"],
    2:["","%50%"],
    3:["","%40%","%75%"],
    4:["","%25%","%50%","%75%"],
    5:["","%25%","%40%", "%60%","%75%"],
    6:["","%20%","%35%","%50%", "%60%","%75%"],
    7:["","%15","%25%","%35%","%50%", "%60%","%75%"],
    8:["","%15","%25%","%35%","%45%","%55%", "%65%","%75%"],
    9:["","%5%","%15","%25%","%35%","%45%","%55%", "%65%","%75%"],
    10:["","%8%","%16%","%24%","%32%","%40%","%48%", "%56%","%64%","%75%"],
}

In [4]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [5]:
from MoodleXML import *
import yaml

In [6]:
yamlFichier=base+theme+"-"+enonce+".yaml"
csvFichier=base+theme+".txt"
boolEX=True
titreNoEX=False

In [7]:
with open(yamlFichier, 'r') as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r', encoding="utf8") as file:
    donneesLignes=file.readlines()

In [8]:
def donneesEX():
    titre=[]
    result={}
    boolMultiLignes=False
    i=0
    for ligne in donneesLignes:
        if debug or 0: print ligne
        i+=1
        if ligne.startswith("#EX;"):
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#EXM;"):
            boolMultiLignes=True
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                titre=temp[1:]            
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            if boolMultiLignes:
                result["boucle"].extend(temp)
            else:
                result["boucle"].append(temp)
#            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
else:
#    print "sans #EX;"
    donneesNoEX()

In [9]:
def lierMot(ref,chaine):
    if motReferences[ref]:
        mot=motReferences[ref]
    else:
        mot=ref
    result=chaine.replace(ref,u'<a href="https://fr.wikipedia.org/wiki/%s" target="_blank" style="color:blue">%s</a>'%(mot,ref))
    return result

def echapperCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\\[\1\\]",corps)
        result=prefixe+corps+suffixe
    print "échappé",result
    return result    

def modifierCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()        
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\\[\1\\]",corps)
        unites=filter(None,corps.split("&&"))
        print "unités",unites
        corps=".*".join(unites)
        result=prefixe+corps+suffixe
    print "échappé",result
    return result    


def separerMots(elements,titre):
    result=[]
    elements = filter(None, elements)
    nbMots=len(elements[0].strip().split(" "))
    solution=u"=%s#Bravo ! La phrase était : %s"%(elements[0],titre)
    corrections=[]
    nbElements=len(elements[1:])
#    if not nbElements in nbIndiceNotes: 
#        print "Ajouter une répartition des notes pour %d indices"%nbElements
#        print titre
#        print elements[1:]
    for n,element in enumerate(elements[1:]):
        (reponseSpec,feedbackSpec)=element.split("#")
        reponseSpec=modifierCorrection(reponseSpec)
        if feedbackSpec:
            for mot in motReferences:
                if mot in feedbackSpec:
                    feedbackSpec=lierMot(mot,feedbackSpec)
        element="#".join([reponseSpec,feedbackSpec])
        corrections.append(nbIndiceNotes(nbElements)[n]+element)
#    print nbMots
    minMots="^"+"([^ ]+ +)?"*(nbMots-4)+"[^ ]+$#%s"%msgPlusDeMots
    maxMots="^"+"([^ ]+ +)"*(nbMots+2)+"([^ ]+ +)*[^ ]+$#%s"%msgMoinsDeMots
#    print "~".join(elements)
    result.extend([solution,minMots,maxMots])
    result.extend(corrections)
    return result

In [10]:
consigne="<br/>".join(configuration[u"Consigne"])
motReferences=configuration[u"Références"]
separationExercices=[]
for question in donnees:
    questionLignes=[]
    for nom in question:
        titre=nom
        questionLignes.append(consigne)
        questionLignes.append(u"".join(re.split("[ -]",question[nom]["boucle"][0])))
#        questionLignes.append(u"".join(question[nom]["boucle"][0].split(" ")))
        questionLignes.append(u"{1:RXC:%s}"%u"~".join(separerMots(question[nom]["boucle"],titre)))
#        print questionLignes
        textQuestion=u"<br/>".join(questionLignes)
        separationExercices.append(ClozeExercice(titre,textQuestion,penalty=adaptatifPenalite))

original --.*(&& ʃa && gʁi && suʁi && o && də && gutjɛʁ && vɛʁ && klɛʁ).*
unités [u' \u0283a ', u' g\u0281i ', u' su\u0281i ', u' o ', u' d\u0259 ', u' gutj\u025b\u0281 ', u' v\u025b\u0281 ', u' kl\u025b\u0281']
échappé --.*( ʃa .* gʁi .* suʁi .* o .* də .* gutjɛʁ .* vɛʁ .* klɛʁ).*
original --.*opsɛʁv-ɛ .*
échappé --.*opsɛʁv-ɛ .*
original --.*(&&l-ə && d-y && l-a ).*
unités [u'l-\u0259 ', u' d-y ', u' l-a ']
échappé --.*(l-ə .* d-y .* l-a ).*
original --.*nu\S*z-opsɛʁv-ɛ .*
échappé --.*nu\S*z-opsɛʁv-ɛ .*
original --.* nu-z-opsɛʁv-ɛ .*
échappé --.* nu-z-opsɛʁv-ɛ .*
original --.* kɥizin
échappé --.* kɥizin
original --.* uvʁə-bwat .*
échappé --.* uvʁə-bwat .*
original --.* œ̃-n .*
échappé --.* œ̃-n .*
original --amɛn-mwa .*
échappé --amɛn-mwa .*
original --.*(&& ɑ̃fɑ̃ && aʁive && lise && bys).*
unités [u' \u0251\u0303f\u0251\u0303 ', u' a\u0281ive ', u' lise ', u' bys']
échappé --.*( ɑ̃fɑ̃ .* aʁive .* lise .* bys).*
original --.*s-ɛt .*
échappé --.*s-ɛt .*
original --.* ɛ-t .*
échappé --.

In [11]:
categorie=configuration[u"Catégorie"]
clozeQuestions=XMLClozes(categorie)

In [12]:
for exercice in separationExercices:
    clozeQuestions.addExercice(exercice)

In [13]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(clozeQuestions.getClozes())

In [14]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [15]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [16]:
motReferences

{'clitique': None,
 'datif': None,
 u'd\xe9monstratif': u'Adjectif_d\xe9monstratif',
 'liaison': u'Liaison_en_fran\xe7ais',
 'partitif': 'Article_partitif',
 'possessif': None,
 u'pr\xe9position': None,
 u'r\xe9fl\xe9chi': u'Morphologie_du_pronom_personnel_en_fran\xe7ais#Pronom_r.C3.A9fl.C3.A9chi_et_pronom_non_r.C3.A9fl.C3.A9chi'}

In [17]:
correction=u"--.*(&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]).*"
#correction=u"--.*(&& DEF-M.SG.⊖ chat&& DEF-F.SG.⊖ gouttière).*"
(prefixe, corps, suffixe)=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",correction).groups()
print corps
subParentheses = re.compile(ur"\(([^)]*)\)")
result=subParentheses.sub(ur"\\(\1\\)",corps)
print result
subCrochets = re.compile(ur"\[([^]]*)\]")
result=subCrochets.sub(ur"\\[\1\\]",result)
subPoints = re.compile(ur"(\.[^*])")
result=subPoints.sub(ur"\\\1",result)

print prefixe+result+suffixe

&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]
&& gris\(M\)[SG.⊖] && vert\(M\)[SG.⊖] && clair[M.SG.⊖]
--.*(&& gris\(M\)\[SG\.⊖\] && vert\(M\)\[SG\.⊖\] && clair\[M\.SG\.⊖\]).*


In [18]:
subParentheses = re.compile(ur"&&([^&)]*)\(([^)]*)\)")
subCrochets = re.compile(ur"&&([^&[]*)\[([^\]]*)\]")
subPoints = re.compile(ur"&&(([^)\.]|\\\)|\\\.)+)(\.[^*])")
print subPoints.search(correction)
result=subParentheses.sub(ur"&&\1\\(\2\\)",correction)
print result
result=subCrochets.sub(ur"&&\1\\[\2\\]",result)
print result
result=subPoints.sub(ur"&&\1\\\3",result)
print result

--.*(&& gris\(M\)[SG.⊖] && vert\(M\)[SG.⊖] && clair[M.SG.⊖]).*
--.*(&& gris\(M\)\[SG.⊖\] && vert\(M\)\[SG.⊖\] && clair\[M.SG.⊖\]).*
--.*(&& gris\(M\)\[SG\.⊖\] && vert\(M\)\[SG\.⊖\] && clair\[M\.SG.⊖\]).*
